In [62]:
import pandas as pd
import ta
from plot_utils import plot_results

In [63]:
df = pd.read_csv('BTC_ETH_LTC_1week_1m.csv')

In [64]:
symbols = ['BTC', 'ETH', 'LTC']



for symbol in symbols:
    df[f'{symbol}_EMA_50'] = ta.trend.ema_indicator(df[f'{symbol}-USD_Close'], 50, fillna=False)
    
    df[f'{symbol}_EMA_200'] = ta.trend.ema_indicator(df[f'{symbol}-USD_Close'], 200, fillna=False)
    
    df[f'{symbol}_ATR'] = ta.volatility.average_true_range(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], window=14)

df.dropna(inplace=True)

In [65]:
df['ETH-USD_Low']

199     2944.67
200     2944.67
201     2947.29
202     2946.61
203     2946.09
         ...   
9956    3376.41
9957    3377.30
9958    3375.76
9959    3382.93
9960    3384.01
Name: ETH-USD_Low, Length: 9762, dtype: float64

In [66]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = 0.99925      # VIP level 0, paying fees with BNB 0.99925
        self.symbols = symbols

        self.buy_price = 0
        self.sell_price = 0

        self.take_profit = 3
        self.stop_loss = 3

        self.wins = 0
        self.losses = 0
        self.up_status = {}
        self.up_candlesticks = {}
        self.reset_up_status([symbols])
        self.reset_up_candlesticks([symbols])

        self.down_status = {}
        self.down_candlesticks = {}
        self.reset_down_status([symbols])
        self.reset_down_candlesticks([symbols])
    
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
        self.buy_price = buy_price

    def sell(self, sell_price, time):
        self.balance_amount = (self.balance_amount * sell_price) * self.trading_fee_multiplier
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'BUSD'

    def reset_up_status(self, symbols):
        for symbol in self.symbols:
            self.up_status[symbol] = 'None'
    
    def reset_up_candlesticks(self, symbols):
        for symbol in self.symbols:
            self.up_candlesticks[symbol] = 0



    def reset_down_status(self, symbols):
        for symbol in self.symbols:
            self.down_status[symbol] = 'None'
    
    def reset_down_candlesticks(self, symbols):
        for symbol in self.symbols:
            self.down_candlesticks[symbol] = 0


In [67]:
'''
Wait for 50 EMA to cross over 200 EMA

Wait for candlestick to hit 50 EMA
    Wait for until we break back over/under 50 EMA
    Make sure it took less than 3 candles to get back to the 50 EMA


Set stop-loss 2x ATR
Set take-profit at 3x ATR
'''

'\nWait for 50 EMA to cross over 200 EMA\n\nWait for candlestick to hit 50 EMA\n    Wait for until we break back over/under 50 EMA\n    Make sure it took less than 3 candles to get back to the 50 EMA\n\n\nSet stop-loss 2x ATR\nSet take-profit at 3x ATR\n'

In [68]:
env = TradingEnv(100, 'BUSD', symbols)
env.up_candlesticks

{'BTC': 0, 'ETH': 0, 'LTC': 0}

In [69]:
for i in range(len(df)):

    if i == 0:
        continue
    if env.balance_unit == 'BUSD':  # Want to buy
        print(env.balance_amount)
        for symbol in symbols:
            # If up_status is buy check if buy is available else add 1 to env.up_candlesticks
            if env.up_status[symbol] == 'Buy':
                if df[f'{symbol}-USD_Close'].iloc[i] >= df[f'{symbol}_EMA_50'].iloc[i]: # Good to buy
                    env.buy(symbol, df[f'{symbol}-USD_Close'].iloc[i], df['CloseTime'].iloc[i])
                    print(symbol, df[f'{symbol}-USD_Close'].iloc[i], df['CloseTime'].iloc[i])
                    env.reset_up_candlesticks([symbol])
                    env.reset_up_status([symbol])
                else: # Not ready to buy
                    env.up_candlesticks[symbol] += 1
                    if env.up_candlesticks[symbol] == 3:
                        env.reset_up_candlesticks([symbol])
                        env.reset_up_status([symbol])
                        

            # Check if up_cross
            if df[f'{symbol}_EMA_50'].iloc[i-1] <= df[f'{symbol}_EMA_200'].iloc[i-1] and df[f'{symbol}_EMA_50'].iloc[i] > df[f'{symbol}_EMA_200'].iloc[i]:
                env.up_status[symbol] = 'Crossed'
                print(f'{symbol} Crossed', df['CloseTime'].iloc[i])
            
            # Checl if up_cross and thouched
            if env.up_status[symbol] == 'Crossed' and df[f'{symbol}-USD_Low'].iloc[i] <= df[f'{symbol}_EMA_50'].iloc[i]:
                env.up_status[symbol] = 'Buy'
                print(f'{symbol} Buy', df['CloseTime'].iloc[i])
    
    else:
        if df[f'{env.balance_unit}-USD_High'].iloc[i] >= env.buy_price + (env.take_profit * df[f'{env.balance_unit}_ATR'].iloc[i]):
            print('Win', env.buy_price + (env.take_profit * df[f'{env.balance_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
            env.sell(env.buy_price + (env.take_profit * df[f'{env.balance_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
            env.wins += 1
            continue
        
        if df[f'{env.balance_unit}-USD_Low'].iloc[i] <= env.buy_price - (env.stop_loss * df[f'{env.balance_unit}_ATR'].iloc[i]):
            print('Loss', env.buy_price - (env.stop_loss * df[f'{env.balance_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
            env.sell(env.buy_price - (env.stop_loss * df[f'{env.balance_unit}_ATR'].iloc[i]), df['CloseTime'].iloc[i])
            env.losses += 1
            
if env.balance_unit != 'BUSD':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit, env.wins, env.losses)

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100


In [70]:
print(env.balance_amount, len(env.buys))
plot_results(df, 'BTC', env.buys, env.sells)


93.96455969896745 41
